In [ ]:
### IMPORTIERE PAKETE
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering, KMeans

from loadmodelingfunctions import generate_loadprofile, build_example_year, visualize_typedays, mape_simulation
import loaddata
import clustertools as ct
import dataprocessing as dp

### AUTORELOAD
%reload_ext autoreload
%autoreload 2

Daten importieren

In [ ]:
stationen=loaddata.load_stations()
display(stationen)
list_power_d, list_power_yd, list_power_ysd, list_power_ymd, list_mat_power_d, list_power_ywd, list_power_yswd, list_power_only_d = loaddata.load_data_from_measurements([0,1,2,3,4,5,6,7,8], d=True, yd=True, ysd=True,ywd=True,yswd=True)

Clusteranalyse

In [ ]:
week_profiles = ct.build_winter_summer_week_profile(list_mat_power_d)
export_plots = False

### OPTIMALE ANZAHL

ct.analyze_cluster_indicators(week_profiles)
if export_plots:
    plt.savefig('export/DBI_CDI.pdf',bbox_inches='tight')

### HIERARCHISCHES CLUSTERN

n_clusters = 5
mymodel_opt = AgglomerativeClustering(n_clusters = n_clusters)
mymodel_opt = mymodel_opt.fit(week_profiles)

Lastprofile

In [ ]:
cluster_names = ['SS','W1','W2','W3','W4']
anzahl_cluster = len(cluster_names)
label=mymodel_opt.labels_

###Generiere Cluster-Daten

all_powers_list = list()
all_powers_list_years = list()

for i in range(0,anzahl_cluster):
    ind=np.zeros(0,int)
    for j in range(0,9):
        if label[j]==i:
            ind=np.append(ind,j)
    list_power = list_power_ysd[ind[0]][2018]
    list_power_years = list_power_yd[ind[0]]
    for j in ind[1:len(ind)]:
        list_power = list_power + list_power_ysd[j][2018]
        list_power_years = list_power_years + list_power_yd[j]
    all_powers_list.insert(len(all_powers_list), list_power)
    all_powers_list_years.insert(len(all_powers_list_years), list_power_years)

In [ ]:
### LASTPROFILE GENERIEREN

load_profiles_list = list()
reg_polynoms_list = list()

for i in range(0,n_clusters):
    load_profiles, coeff_regression = generate_loadprofile(all_powers_list[i],all_powers_list_years[i],plots=False)
    load_profiles_list.insert(len(load_profiles_list), load_profiles)
    reg_polynoms_list.insert(len(reg_polynoms_list), coeff_regression)

In [ ]:
### LASTPROFILE VISUALISIEREN

no=4
profil=load_profiles_list[no]*1000
titel = 'Stationsgruppe' + cluster_names[no]
shorttitle = cluster_names[no]

visualize_typedays(profil, cluster_names[no], export_plots = False, filename='')